In [0]:
%run ./setAccess

In [0]:
# create delta tables in 'curated/dashboard/' layer using parquet files from 'curated/'
from delta.tables import *

def readTable(fileName,dir,type):
    return (spark.read.format(type)
            .option("header", "true")
            .option("inferSchema", "true")
            .load("abfss://"+container+"@"+storageAccountName+".dfs.core.windows.net/"+
                  dirs[dir]+fileName+"."+type)
           )

def readParquetSaveDelta(fileName):
    table = readTable(fileName,"lvl3","parquet")
    dbutils.fs.rm("abfss://"+container+"@"+storageAccountName+".dfs.core.windows.net/"+
                  dirs["lvl3-dashboard"]+fileName+".delta",recurse=True)
    table.write.format("delta").save("abfss://"+container+"@"+storageAccountName+".dfs.core.windows.net/"+
                                     dirs["lvl3-dashboard"]+fileName+".delta")
    return readTable(fileName,"lvl3-dashboard","delta")

factsTable = readParquetSaveDelta("airlines")
dimDate = readParquetSaveDelta("dimDate")
dimTimes = readParquetSaveDelta("dimTimes")
dimFlight = readParquetSaveDelta("dimFlight")
dimDelay = readParquetSaveDelta("dimDelay")

In [0]:
# simple dashboard
from pyspark.sql.functions import *
join = (factsTable.join(dimFlight,factsTable["dimFlight_ID"]==dimFlight["ID"],"inner")
                  .join(dimDate,factsTable["dimDate_ID"]==dimDate["ID"],"inner")
                  .select("Year","Month","Unique_carrier","Flight_num","Origin","Dest","Distance")
       )
join.printSchema()
display(join)
display(join.groupBy("Unique_carrier","Flight_num").sum("Distance").withColumnRenamed("sum(Distance)","Sum_distance"))
display(join.groupBy("Year","Unique_carrier").sum("Distance"))
display(join.groupBy("Year","Month")
             .agg(
                 sum("Distance").alias("Sum_distance"),
                 avg("Distance"),
                 max("Distance"),
                 min("Distance"))
             .sort(col("Sum_distance").desc())
       )
display(join.groupBy("Unique_carrier","Flight_num")
            .min("Distance").withColumnRenamed("min(Distance)","Min_distance")
            .where((col("Min_distance") > 13) & (col("Min_distance") < 17))
            .sort("Min_distance")            
       )